In [8]:
import json, csv, re, argparse, sys, math, random
from pathlib import Path
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError
import pickle

from PIL import Image, ImageDraw, ImageFont
import textwrap
import unicodedata
import requests
from io import BytesIO

In [ ]:
# =================== 全域設定 ===================
USE_MODEL = True
BLEND = 0.2
DRAW_DEBUG_BOXES = False
PRINT_FIRST_PRED = True

# Label scaling（對應 tanh 輸出），推論要乘回
DX_DY_SCALE = 0.1
DLOG_SCALE  = 0.25

EXCEL_PATH = "Board Click SKU.xlsx"
SHEET_USER = "User_Input"
SHEET_IMG  = "SKU_Image"
SHEET_ICON = "SKU_Icon"

TEMPLATE_PATH = "sasa_pink_1280.png"
INITIAL_BOXES_PATH = "Initial_boxes.json"

MODEL_PATH = "layout_model.keras"
SCALER_PATH = "scaler.pkl"

OUT_CSV = "offsets_for_tf.csv"
CLASS_ORDER_FILE = Path(OUT_CSV).with_suffix(".classes.json")  # [NEW]

OUTPUT_DIR = Path("out_cards_cn")
OUTPUT_DIR.mkdir(exist_ok=True)

MAP_TEXT = {
    "Brand Position":          "Brand",
    "VIP Price Position":      "VIP價",
    "Non VIP Price Position":  "優惠價",
    "Original Price Position": "建議價",
    "Product Position":        "Product name",
    "Sales Period Position":   "優惠期",
    "FAB Position":            "FAB",
}
COL_PRODUCT_SKU = "Product SKU"
COL_ICON_NAME   = "Icon"

SKU_IMG_SKU_COL   = "Product SKU"
SKU_IMG_LINK_COL  = "Image"
SKU_ICON_NAME_COL = "Icon"
SKU_ICON_LINK_COL = "Icon Image"

# 字型
FONT_REGULAR_PATH = Path.home() / "Library/Fonts/NotoSansCJKsc-Regular.otf"
FONT_BOLD_PATH    = Path.home() / "Library/Fonts/NotoSansCJKsc-Bold.otf"

# =================== 工具 ===================
def safe_str(x): 
    return "" if (x is None or (isinstance(x,float) and pd.isna(x))) else str(x)

def count_digits(s): 
    return sum(ch.isdigit() for ch in safe_str(s))

def read_json_file(path: Path):
    text = path.read_text(encoding="utf-8").strip()
    return json.loads(text)

def parse_ls_rect(item):
    v = item["value"]
    label = v["rectanglelabels"][0] if isinstance(v.get("rectanglelabels"), list) else v.get("labels", [""])[0]
    rect_pct = dict(x=v["x"], y=v["y"], w=v["width"], h=v["height"])
    ow = item.get("original_width") or item.get("image_original_width")
    oh = item.get("original_height") or item.get("image_original_height")
    return label, rect_pct, int(ow), int(oh)

def pct_to_px(rpct, ow, oh):
    return {"x": rpct["x"]/100.0*ow, "y": rpct["y"]/100.0*oh, "w": rpct["w"]/100.0*ow, "h": rpct["h"]/100.0*oh}

def resize_rect(rect, from_w, from_h, to_w, to_h):
    sx, sy = to_w/float(from_w), to_h/float(from_h)
    return {"x": rect["x"]*sx, "y": rect["y"]*sy, "w": rect["w"]*sx, "h": rect["h"]*sy}

def delta_from(gt, init, W, H):
    w0 = max(1e-6, float(init["w"]))
    h0 = max(1e-6, float(init["h"]))
    return {
        "dx":   (float(gt["x"]) - float(init["x"])) / float(W),
        "dy":   (float(gt["y"]) - float(init["y"])) / float(H),
        "dlogw": float(np.log(max(1e-6,float(gt["w"])) / w0)),
        "dlogh": float(np.log(max(1e-6,float(gt["h"])) / h0)),
    }

def _load_font(path, size): return ImageFont.truetype(str(path), size)
def _safe(s): return "" if (s is None or (isinstance(s,float) and pd.isna(s))) else str(s)
def _count_digits(s): return sum(ch.isdigit() for ch in _safe(s))

def _norm_key(x):
    s = "" if pd.isna(x) else str(x)
    s = unicodedata.normalize("NFKC", s)
    return s.strip().casefold()

def _drive_share_to_direct(u: str) -> str:
    if not u: return u
    m = re.search(r"/d/([A-Za-z0-9_-]+)", u)
    if m: return f"https://drive.google.com/uc?export=download&id={m.group(1)}"
    m = re.search(r"[?&]id=([A-Za-z0-9_-]+)", u)
    if m: return f"https://drive.google.com/uc?export=download&id={m.group(1)}"
    return u

def load_image_any(path_or_url: str):
    try:
        s = str(path_or_url).strip()
        if s.startswith("http"):
            resp = requests.get(_drive_share_to_direct(s), timeout=15)
            resp.raise_for_status()
            return Image.open(BytesIO(resp.content)).convert("RGBA")
        p = Path(s)
        if p.exists(): return Image.open(p).convert("RGBA")
    except Exception as e:
        print("[WARN] cannot load image:", path_or_url, e)
    return None

def paste_image_into_box(canvas_rgba, path_or_url, box, padding=6):
    im = load_image_any(path_or_url)
    if im is None: return
    x,y,w,h = int(box["x"]), int(box["y"]), int(box["w"]), int(box["h"])
    w2,h2 = max(1,w-padding*2), max(1,h-padding*2)
    ratio = min(w2/im.width, h2/im.height)
    im = im.resize((max(1,int(im.width*ratio)), max(1,int(im.height*ratio))), Image.LANCZOS)
    ox = x+(w-im.width)//2
    oy = y+(h-im.height)//2
    canvas_rgba.alpha_composite(im,(ox,oy))

def draw_text_in_box(draw,text,box,font_path,max_font=64,min_font=16,align="left",line_spacing=1.15):
    if not text or str(text).strip()=="": return
    x,y,w,h = int(box["x"]),int(box["y"]),int(box["w"]),int(box["h"])
    text = str(text)
    for fs in range(max_font,min_font-1,-2):
        font = _load_font(font_path, fs)
        approx_chars = max(1,int(w/(fs*0.55)))
        lines=[]
        for raw in text.split("\n"):
            lines += (textwrap.wrap(raw,width=approx_chars) if approx_chars>1 else [raw])
        bboxes=[draw.textbbox((0,0),ln,font=font) for ln in lines]
        total_h=int(sum(bb[3]-bb[1] for bb in bboxes)+(len(lines)-1)*fs*(line_spacing-1))
        if total_h<=h:
            cur_y=y+(h-total_h)//2
            for ln in lines:
                bb=draw.textbbox((0,0),ln,font=font)
                lw=bb[2]-bb[0]
                cur_x=x if align=="left" else (x+(w-lw)//2 if align=="center" else x+(w-lw))
                draw.text((cur_x,cur_y),ln,font=font,fill=(0,0,0,255))
                cur_y+=int(fs*line_spacing)
            return

def _stroke(draw, box, color=(0,0,0,255), width=2):
    x,y,w,h = int(box["x"]),int(box["y"]),int(box["w"]),int(box["h"])
    draw.rectangle([x,y,x+w,y+h], outline=color, width=width)

def _clip(v, lo, hi): return max(lo, min(hi,v))

In [ ]:
# =================== 特徵 ===================
FEATURE_ORDER = ["title_len","bullets_len","bullets_lines","vip_digits","unit_digits","nonvip_digits","has_badge"]

def build_features(meta_row: dict):
    return {
        "title_len": len(safe_str(meta_row.get("title",""))),
        "bullets_len": len(safe_str(meta_row.get("bullets",""))),
        "bullets_lines": len([b for b in re.split(r"[;\n]", safe_str(meta_row.get("bullets",""))) if b.strip()]),
        "vip_digits": count_digits(meta_row.get("vip_price","")),
        "unit_digits": count_digits(meta_row.get("unit_price","")),
        "nonvip_digits": count_digits(meta_row.get("non_vip_price","")),
        "has_badge": int(str(meta_row.get("has_badge",0)).strip() not in ["","0","False","false"]),
    }

def build_features_from_row(row: pd.Series, resolved_icon_path: str) -> dict:
    return {
        "title_len": len(_safe(row.get(MAP_TEXT["Product Position"],""))),
        "bullets_len": len(_safe(row.get(MAP_TEXT["FAB Position"],""))),
        "bullets_lines": len([b for b in _safe(row.get(MAP_TEXT["FAB Position"],"")).split(";") if b.strip()]),
        "vip_digits": _count_digits(row.get(MAP_TEXT["VIP Price Position"],"")),
        "unit_digits": _count_digits(row.get(MAP_TEXT["Non VIP Price Position"],"")),
        "nonvip_digits": _count_digits(row.get(MAP_TEXT["Non VIP Price Position"],"")),  # [FIX]
        "has_badge": int(Path(_safe(resolved_icon_path)).exists()) if resolved_icon_path else 0,
    }

# =================== Preprocess → CSV ===================
def run_preprocess(label_studio_json: Path, initial_boxes_json: Path, out_csv: Path, meta_csv: Path|None=None):
    init_cfg = read_json_file(initial_boxes_json)
    Tw, Th = int(init_cfg["canvas"]["width"]), int(init_cfg["canvas"]["height"])
    init_boxes = init_cfg["boxes"]
    classes = list(init_boxes.keys())

    tasks = read_json_file(label_studio_json)
    rows=[]
    for t in tasks:
        annos = t.get("annotations") or []
        if not annos: continue
        res = annos[0].get("result", [])
        gt_scaled={}
        for r in res:
            if r.get("type") not in ("rectanglelabels","rectangles"): continue
            label, rpct, ow, oh = parse_ls_rect(r)
            if label not in classes: continue
            rect_px = pct_to_px(rpct, ow, oh)
            rect_tpl = resize_rect(rect_px, ow, oh, Tw, Th)
            gt_scaled[label]=rect_tpl
        row={"image_id":t.get("id")}
        feats=build_features({})
        row.update(feats)
        for cls in classes:
            init=init_boxes[cls]; gt=gt_scaled.get(cls)
            d=delta_from(gt,init,Tw,Th) if gt else {"dx":0,"dy":0,"dlogw":0,"dlogh":0}
            for k,v in d.items(): row[f"{cls}_{k}"]=v
        rows.append(row)
    df=pd.DataFrame(rows)
    df.to_csv(out_csv,index=False)
    class_file = out_csv.with_suffix(".classes.json")
    json.dump(classes, open(class_file, "w", encoding="utf-8"), ensure_ascii=False, indent=2) # [NEW]
    print(f"[OK] Saved {out_csv} rows={len(df)}")
    return df

# =================== Augmentation ===================
def augment_offsets(df, classes, num_aug=10):
    aug_rows=[]
    for _, row in df.iterrows():
        for _ in range(num_aug):
            new_row=row.copy()
            for cls in classes:
                new_row[f"{cls}_dx"]=row[f"{cls}_dx"]+random.uniform(-0.15,0.15)
                new_row[f"{cls}_dy"]=row[f"{cls}_dy"]+random.uniform(-0.15,0.15)
                new_row[f"{cls}_dlogw"]=row[f"{cls}_dlogw"]+random.uniform(-0.2,0.2)
                new_row[f"{cls}_dlogh"]=row[f"{cls}_dlogh"]+random.uniform(-0.2,0.2)
            aug_rows.append(new_row)
    return pd.concat([df,pd.DataFrame(aug_rows)],ignore_index=True)

# =================== 訓練（tanh + label scaling） ===================
def train_model():
    df=pd.read_csv(OUT_CSV)
    classes=json.load(open(CLASS_ORDER_FILE, "r", encoding="utf-8"))
    df_aug=augment_offsets(df,classes,num_aug=10)
    print(f"[INFO] Augmented {len(df)}→{len(df_aug)}")
    X=df_aug.drop(columns=[c for c in df_aug.columns if c.endswith(("_dx","_dy","_dlogw","_dlogh")) or c=="image_id"])
    Y=df_aug[[c for c in df_aug.columns if c.endswith(("_dx","_dy","_dlogw","_dlogh"))]]
    Y_scaled=Y.copy()
    for c in Y.columns:
        if c.endswith(("_dx","_dy")): Y_scaled[c]=Y[c]/DX_DY_SCALE
        else: Y_scaled[c]=Y[c]/DLOG_SCALE
    Xtr,Xv,Ytr,Yv=train_test_split(X,Y_scaled,test_size=0.2,random_state=42)
    scaler=StandardScaler(); Xtr=scaler.fit_transform(Xtr); Xv=scaler.transform(Xv)
    model=tf.keras.Sequential([
        tf.keras.layers.Dense(128,activation='relu',input_shape=(Xtr.shape[1],)),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dense(Ytr.shape[1],activation='tanh')  # [NEW]
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),loss='mse',metrics=[MeanSquaredError(),MeanAbsoluteError()])
    model.fit(Xtr,Ytr,validation_data=(Xv,Yv),epochs=50,batch_size=16,verbose=1)
    model.save(MODEL_PATH); pickle.dump(scaler,open(SCALER_PATH,"wb"))
    print("[OK] Model saved")

# =================== 幾何 / 佈局 ===================
def apply_offset(init_box, delta, W, H):
    dx, dy, dlogw, dlogh = map(float, delta)
    # （可選）安全範圍限制
    dx = _clip(dx, -0.5, 0.5)          # [NEW] 
    dy = _clip(dy, -0.5, 0.5)          # [NEW]
    sx = math.exp(_clip(dlogw, math.log(0.5), math.log(2.0)))  # [NEW]
    sy = math.exp(_clip(dlogh, math.log(0.5), math.log(2.0)))  # [NEW]
    x = init_box["x"] + dx * W
    y = init_box["y"] + dy * H
    w = max(24, init_box["w"] * sx)
    h = max(24, init_box["h"] * sy)
    x = _clip(x, 0, W - w)             # [NEW] 
    y = _clip(y, 0, H - h)             # [NEW]
    return {"x": x, "y": y, "w": w, "h": h}

# =================== 模型 + Scaler 載入 ===================
def load_template_and_boxes():
    tpl = Image.open(TEMPLATE_PATH).convert("RGBA")
    tw, th = tpl.size
    init_cfg = json.loads(Path(INITIAL_BOXES_PATH).read_text(encoding="utf-8"))
    cw, ch = init_cfg["canvas"]["width"], init_cfg["canvas"]["height"]
    if (tw, th) != (cw, ch):
        sx, sy = tw/float(cw), th/float(ch)
        for _, r in init_cfg["boxes"].items():
            r["x"], r["y"] = r["x"]*sx, r["y"]*sy
            r["w"], r["h"] = r["w"]*sx, r["h"]*sy
        init_cfg["canvas"]["width"], init_cfg["canvas"]["height"] = tw, th
        print(f"[INFO] Scaled initial boxes to {tw}x{th}")
    return tpl, init_cfg

def load_model_and_scaler():
    model = tf.keras.models.load_model(MODEL_PATH) if Path(MODEL_PATH).exists() else None
    scaler = pickle.load(open(SCALER_PATH,"rb")) if Path(SCALER_PATH).exists() else None
    if model is None:  print("[WARN] MODEL not found → zero offsets")
    if scaler is None: print("[WARN] SCALER not found → use raw features")
    return model, scaler


In [ ]:
# =================== 渲染主程式 ===================
def main_render():
    xls = pd.ExcelFile(EXCEL_PATH)
    df_user = pd.read_excel(xls, sheet_name=SHEET_USER)
    df_img  = pd.read_excel(xls, sheet_name=SHEET_IMG)
    df_icon = pd.read_excel(xls, sheet_name=SHEET_ICON)

    # 建查表
    img_lookup  = {_norm_key(r[SKU_IMG_SKU_COL]): str(r[SKU_IMG_LINK_COL]).strip() for _, r in df_img.iterrows()}
    icon_lookup = {_norm_key(r[SKU_ICON_NAME_COL]): str(r[SKU_ICON_LINK_COL]).strip() for _, r in df_icon.iterrows()}

    # 載入模板與模型
    template_rgba, init_cfg = load_template_and_boxes()
    W, H = init_cfg["canvas"]["width"], init_cfg["canvas"]["height"]
    INIT_BOXES = init_cfg["boxes"]

    # [FIX] 讀回訓練時類別順序，並檢查集合一致
    if CLASS_ORDER_FILE.exists():
        train_classes = json.load(open(CLASS_ORDER_FILE, "r", encoding="utf-8"))
        if set(train_classes) != set(INIT_BOXES.keys()):
            raise ValueError(f"Classes mismatch.\nTrain: {train_classes}\nInfer: {list(INIT_BOXES.keys())}")
        CLASSES = train_classes[:]
    else:
        CLASSES = list(INIT_BOXES.keys())

    model, scaler = load_model_and_scaler()

    # 逐列生成
    for idx, row in df_user.iterrows():
        sku_key  = _norm_key(row.get(COL_PRODUCT_SKU, ""))
        icon_key = _norm_key(row.get(COL_ICON_NAME, ""))
        prod_img_path = img_lookup.get(sku_key, "")
        icon_img_path = icon_lookup.get(icon_key, "")

        # 特徵 → DataFrame 給 scaler（避免 sklearn warning）
        feats = build_features_from_row(row, resolved_icon_path=icon_img_path)
        X_df = pd.DataFrame([feats], columns=FEATURE_ORDER).astype(float)  # [FIX]
        X_in = scaler.transform(X_df) if scaler is not None else X_df.values

        # baseline offsets (全 0)
        deltas_base = {cls: [0.0,0.0,0.0,0.0] for cls in CLASSES}

        # 預測 offsets（tanh 輸出需乘回 label scaling）
        if USE_MODEL and model is not None:
            pred = model.predict(X_in, verbose=0)[0]
            deltas_pred = {}
            for i, cls in enumerate(CLASSES):
                dx = float(pred[i*4+0]) * DX_DY_SCALE
                dy = float(pred[i*4+1]) * DX_DY_SCALE
                dw = float(pred[i*4+2]) * DLOG_SCALE
                dh = float(pred[i*4+3]) * DLOG_SCALE
                deltas_pred[cls] = [dx,dy,dw,dh]
        else:
            deltas_pred = deltas_base

        if idx == 0 and PRINT_FIRST_PRED and USE_MODEL and (model is not None):  # [FIX]
            print("Pred deltas (first row, scaled back):", pred[:min(12, len(pred))])

        # blend baseline 與 model
        deltas_final = {}
        for cls in CLASSES:
            db = deltas_base[cls]; dp = deltas_pred[cls]
            deltas_final[cls] = [(1-BLEND)*db[i] + BLEND*dp[i] for i in range(4)]

        final_boxes = {cls: apply_offset(INIT_BOXES[cls], deltas_final[cls], W, H) for cls in CLASSES}

        # 繪圖
        canvas = template_rgba.copy()
        draw = ImageDraw.Draw(canvas)

        paste_image_into_box(canvas, prod_img_path, final_boxes["Product Image Position"])
        paste_image_into_box(canvas, icon_img_path,  final_boxes["Icon Position"])

        draw_text_in_box(draw, row.get(MAP_TEXT["Brand Position"], ""),          final_boxes["Brand Position"],          font_path=FONT_BOLD_PATH,   max_font=72)
        draw_text_in_box(draw, row.get(MAP_TEXT["Product Position"], ""),        final_boxes["Product Position"],        font_path=FONT_REGULAR_PATH,max_font=64)
        fab_text = str(row.get(MAP_TEXT["FAB Position"], "") or "").replace(";", "\n")
        draw_text_in_box(draw, fab_text,                                        final_boxes["FAB Position"],            font_path=FONT_REGULAR_PATH,max_font=44)
        draw_text_in_box(draw, row.get(MAP_TEXT["VIP Price Position"], ""),      final_boxes["VIP Price Position"],      font_path=FONT_BOLD_PATH,   max_font=100)
        draw_text_in_box(draw, row.get(MAP_TEXT["Non VIP Price Position"], ""),  final_boxes["Non VIP Price Position"],  font_path=FONT_REGULAR_PATH,max_font=36)
        draw_text_in_box(draw, row.get(MAP_TEXT["Original Price Position"], ""), final_boxes["Original Price Position"], font_path=FONT_REGULAR_PATH,max_font=32)
        draw_text_in_box(draw, row.get(MAP_TEXT["Sales Period Position"], ""),   final_boxes["Sales Period Position"],   font_path=FONT_REGULAR_PATH,max_font=28, align="right")

        if DRAW_DEBUG_BOXES:
            for cls, box in final_boxes.items():
                _stroke(draw, box, (255,0,0,180), 2)

        out_path = OUTPUT_DIR / f"card_cn_{idx+1:03d}.png"
        canvas.convert("RGB").save(out_path, quality=95)
        print("Saved:", out_path)

# =================== 主程式 ===================
if __name__ == "__main__":
    run_preprocess(Path("Objects_positions.json"), Path(INITIAL_BOXES_PATH), Path(OUT_CSV), meta_csv=None)
    train_model()
    main_render()

[OK] Saved offsets_for_tf.csv rows=29
[INFO] Augmented 29→319
Epoch 1/50


/opt/anaconda3/envs/boardclick/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 1.6977 - mean_absolute_error: 1.0058 - mean_squared_error: 1.6977 - val_loss: 1.8264 - val_mean_absolute_error: 1.0306 - val_mean_squared_error: 1.8264
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.6808 - mean_absolute_error: 1.0008 - mean_squared_error: 1.6808 - val_loss: 1.8088 - val_mean_absolute_error: 1.0253 - val_mean_squared_error: 1.8088
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.6645 - mean_absolute_error: 0.9959 - mean_squared_error: 1.6645 - val_loss: 1.7915 - val_mean_absolute_error: 1.0202 - val_mean_squared_error: 1.7915
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.6485 - mean_absolute_error: 0.9912 - mean_squared_error: 1.6485 - val_loss: 1.7747 - val_mean_absolute_error: 1.0152 - val_mean_squared_error: 1.7747
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.6331 - mean_absolute_error: 0.9865 - mean_squared_error: 1.6331 - val_loss: 1.7582 - val_mean_absolute_error: 1